## Generating and Visualizing Antipodal Grasps on Book

In [87]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
import numpy as np
import trimesh
import sys
from typing import List 
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RigidTransform,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    Context,
    Diagram,
    PointCloud,
    Simulator,
    TrajectorySource,
    Solve,
    RotationMatrix,
    MultibodyPlant,
    eq,
    StateInterpolatorWithDiscreteDerivative,
    MinimumDistanceLowerBoundConstraint,
    RollPitchYaw,
    SolutionResult,
    CollisionFilterDeclaration,
    GeometrySet,
    Role,
)
from pydrake.geometry import Meshcat
from pydrake.multibody import inverse_kinematics

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb, AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import ConfigureParser
from manipulation.clutter import GraspCandidateCost, GenerateAntipodalGraspCandidate
from manipulation.icp import IterativeClosestPoint
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
# Own utils
from hwstation.utils import * 
from hwstation.add_objects import *
from perception.icp import run_table_multi_book_icp, check_quality_of_icp
from rrt.rrt_planning import *
from grasp.grasping import *

from enum import Enum
import pandas as pd

from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.station import AddPointClouds
from random import random
from manipulation.pick import *
#from manipulation.pick import *

## 1. Setup environment

In [3]:
# Start meshcat
try:
    meshcat = Meshcat(7010)
except:
    pass #This error appears if this cell is executed twice (port 7006 is already taken then)

scenario_data = get_library_scenario_data()
diagram_plan, visualizer, simulator = init_diagram(meshcat, scenario_data)
fix_input_port(diagram_plan, simulator)
diagram_plan_context = diagram_plan.CreateDefaultContext()
scene_graph = diagram_plan.GetSubsystemByName("station").GetSubsystemByName("scene_graph")
sg_context = scene_graph.GetMyContextFromRoot(diagram_plan_context)
filterCollsionGeometry(scene_graph, sg_context)
q0_home = get_iiwa_joint_state(diagram_plan, diagram_plan_context)
simulator.AdvanceTo(1.0)
# diagram_plan_context = simulator.get_mutable_context()
# q0_init = get_iiwa_joint_state(diagram_plan, diagram_plan_context)


INFO:drake:Meshcat listening for connections at http://localhost:7010


Station input port size:  <pydrake.systems.framework.InputPort object at 0x7f91dbc1d9f0>
plant.GetStateNames():  102
plant.GetActuatorNames():  12
Fixing input port of size:  20
Fixing input port of size:  1


==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



In [4]:
# shelf_labels = ["A","B"] #,"C","D","E","F"]
# shelf_pos_list = [(1,1),(0, 0)]#(0,0)]
# N_BOOKS_TO_GRASP = 2
# book_grasp_dur = 10.0
# Q_TRAJECTORY_FPATH = "trajectories/2_q_book_traj"
# WSG_TRAJECTORY_FPATH = "trajectories/2_wsg_book_traj"

In [76]:
shelf_labels = ["A","B" ,"C","D","E","F"]
shelf_pos_list = [(1,1), (2,1), (0,0), (0,1), (1,0), (2,0)]
N_BOOKS_TO_GRASP = 6
book_grasp_dur = 10.0
Q_TRAJECTORY_FPATH = "trajectories/6_q_book_traj"
WSG_TRAJECTORY_FPATH = "trajectories/6_wsg_book_traj"

In [33]:
time_point_list = [(None, 0.0)]
for idx in range(N_BOOKS_TO_GRASP):
    end_time = time_point_list[-1][1]
    book_grasp_time = end_time + book_grasp_dur
    
    shelf_placement_dur = 20.0

    if shelf_labels[idx] in ["B", "C"]:
        shelf_placement_dur = 30.0

    if shelf_labels[idx] in ["D", "E"]:
        shelf_placement_dur = 40.0

    if shelf_labels[idx] == "F":
        shelf_placement_dur = 60.0

    book_pl_time = book_grasp_time + shelf_placement_dur
    time_point_list.append((book_grasp_time,book_pl_time))

## 2. Running ICP to find point cloud

In [7]:
icp_cloud_list, X_MS_hat_list = run_table_multi_book_icp(diagram_plan, diagram_plan_context, meshcat,icp_limit=N_BOOKS_TO_GRASP-1)
for idx, X_MS_hat in enumerate(X_MS_hat_list):
    visualize_frame(str(idx),meshcat, X_MS_hat)

fixing improper rotation
fixing improper rotation
Retrying fitting idx=2
fixing improper rotation
fixing improper rotation


## 3. Trajectory Planning with RRT

In [8]:
# what's different about this function is that now we try to line up with the book but be away from the table
def find_book_grasp_with_y_offset_iiwa_config(icp_cloud: PointCloud, manipSim: ManipulationStationSim, diagram: Diagram, diagram_context: Context, q0_init: np.ndarray, gripper_setpoint: float):
    found_non_collision_points = False
    while True:
        manipSim.SetStationConfiguration(q0_init, gripper_setpoint)
        optim_grasp = sample_grasps(icp_cloud, diagram, diagram_context, n_samples=1000)
        R_optim = optim_grasp.rotation()
        trans_optim = optim_grasp.translation() + np.array([0.0, 0.5, 0.0])
        back_grasp = RigidTransform(R_optim, trans_optim)

        #q0 = get_iiwa_joint_state(diagram, diagram_context)
        q0 = q0_home.copy()
        ik_solver = IKSolver()
        q_goal, optimal = ik_solver.solve(back_grasp, q_guess=q0)
        start_collision = manipSim.ExistsCollision(q0,gripper_setpoint)
        final_collision = manipSim.ExistsCollision(q_goal,gripper_setpoint)
        if (not start_collision) and (not final_collision):
            break
    return q0, q_goal, optim_grasp

In [9]:
def find_path_to_book(icp_cloud: PointCloud, diagram: Diagram, diagram_context: Context, q0_curr: np.ndarray, max_rrt_iter: int = 500):
    gripper_setpoint = 0.1
    manipSim = ManipulationStationSim(diagram, diagram_context)

    path_found = False 
    while not path_found:
        q0, q_goal, optim_grasp = find_book_grasp_with_y_offset_iiwa_config(icp_cloud, manipSim, diagram, diagram_context, q0_curr,gripper_setpoint)
        q0 = q0_home.copy()
        # the goal here is to line up with the book
        
        # after lining up with the book, try to go into the optimal grasp with a bit of bias of going "into" the book
        R_optim = optim_grasp.rotation()
        trans_optim = optim_grasp.translation() + np.array([-0.0, -0.05, -0.025])
        further_grasp = RigidTransform(R_optim, trans_optim)
        ik_solver = IKSolver()
        q_further, optimal = ik_solver.solve(further_grasp, q_guess=q_goal)
        further_collision = manipSim.ExistsCollision(q_further,gripper_setpoint)
        
        if further_collision:
                print("RESTART RRT due to going into grasp colliding")
                continue
        
        iiwa_problem = IiwaProblem(
            q_start=q0,
            q_goal=q_goal,
            gripper_setpoint=gripper_setpoint,
            collision_checker=manipSim,
            debug=True,
            x_limits=(0.0,2.0),
            y_limits=(0.0,2.0),
        )

        path = rrt_planning(iiwa_problem, max_rrt_iter, 0.20)
        if path is None or len(path) > 20: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
            print("RESTART RRT")
            continue

        further_iiwa_problem = IiwaProblem(
            q_start=q_goal,
            q_goal=q_further,
            gripper_setpoint=gripper_setpoint,
            collision_checker=manipSim,
            debug=True,
            x_limits=(0.0,2.0),
            y_limits=(0.0,2.0),
        )
        further_path = rrt_planning(further_iiwa_problem, 50, 1.0)
        if further_path is None or len(further_path) > 20: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
            print("RESTART RRT due to further path failing")
            continue

        path.extend(further_path)
        
        path_found = True
    return path

# Configure this path and obtain joint and wsg_trajectories
def configure_grasping_path(path_arr, start_time, end_time):
    path_arr = np.array(path_arr).T
    wait_at_grasp = len(path_arr[0])
    grasp_pose = path_arr[:, -1]
    grasp_pose = np.tile(grasp_pose, (wait_at_grasp,1)).T
    reversed_path = path_arr[:,::-1]

    # arrive at grasp pose and wait
    complete_path = np.concatenate([path_arr,grasp_pose],axis=1)

    # return from grasp pose
    complete_path = np.concatenate([complete_path,reversed_path],axis=1)

    t_lst = np.linspace(start_time, end_time, complete_path.shape[1])
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, complete_path)

    pre_pick_time_idx = len(path_arr[0])
    post_pick_time_idx = (int)(pre_pick_time_idx + wait_at_grasp)

    opened = np.array([0.1])
    closed = np.array([0.0])

    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
        [t_lst[0], t_lst[pre_pick_time_idx]],
        np.hstack([[opened], [opened]]),
    )
    traj_wsg_command.AppendFirstOrderSegment(t_lst[post_pick_time_idx], closed)
    traj_wsg_command.AppendFirstOrderSegment(t_lst[-1], closed)

    return q_traj, traj_wsg_command

def get_book_grasping_trajectory(book_icp_cloud: PointCloud, diagram: Diagram, diagram_context: Context, q0_curr: np.ndarray, start_time: float, end_time: float, max_rrt_iter: int = 500):
    path = find_path_to_book(book_icp_cloud, diagram, diagram_context, q0_home.copy(), max_rrt_iter=max_rrt_iter)
    q_traj_book_grasp, traj_wsg_book_grasp = configure_grasping_path(path, start_time, end_time)
    return q_traj_book_grasp, traj_wsg_book_grasp
    

In [10]:
from perception.icp import *

In [11]:
q_traj_grasp_list = []
traj_wsg_grasp_list = []
#N_BOOKS_TO_GRASP = 2
book_grasping_time = 10.0
for idx in range(N_BOOKS_TO_GRASP):
    print("Finding trajectory to book: ", idx)
    q_traj_book_grasp, traj_wsg_book_grasp = get_book_grasping_trajectory(icp_cloud_list[idx],diagram_plan, diagram_plan_context, q0_home.copy(), start_time=time_point_list[idx][1], end_time=time_point_list[idx+1][0], max_rrt_iter=100)
    q_traj_grasp_list.append(q_traj_book_grasp)
    traj_wsg_grasp_list.append(traj_wsg_book_grasp)

Finding trajectory to book:  0
Found  573  grasp candidates.
RESTART RRT due to going into grasp colliding
Found  577  grasp candidates.
RESTART RRT due to going into grasp colliding
Found  555  grasp candidates.
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
RESTART RRT
Found  578  grasp candidates.
RESTART RRT due to going into grasp colliding
Found  579  grasp candidates.
RESTART RRT due to going into grasp colliding
Found  541  grasp candidates.
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
Finding trajectory to book:  1
Found  550  grasp candidates.
RESTART RRT due to going into grasp colliding
Found  578  grasp candidates.
RESTART RRT due to going into grasp colliding
Found  569  grasp candidates.
self.collide:  False
self.valid configuration:  True
self.coll

In [12]:
# diagram_plan_context = simulator.get_mutable_context()

In [13]:
#DEBUG
from pydrake.all import StackedTrajectory, CompositeTrajectory
q_traj_book_grasp = q_traj_grasp_list[0]
traj_wsg_book_grasp = traj_wsg_grasp_list[0]

In [14]:
scenario_data = get_library_scenario_data(cameras=False)
meshcat.Delete()
builder = DiagramBuilder()
scenario = load_scenario(data=scenario_data)
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat,parser_preload_callback=ConfigureParser))


# COMMENTING OUT THE POINT CLOUD STUFF FOR NOW FOR THE PURPOSE OF VISUALIZING
# # Adding point cloud extractors:
# to_point_cloud = AddPointClouds(
#     scenario=scenario, station=station, builder=builder, meshcat=meshcat
# )

# # Connect point clouds with output port:
# for idx, name in enumerate(to_point_cloud.keys()):
#     builder.ExportOutput(
#         to_point_cloud[name].get_output_port(), name+"_ptcloud")

builder.ExportOutput(
    station.GetOutputPort("mobile_iiwa.state_estimated"),
    "mobile_iiwa.state_estimated"
)

# builder.ExportInput(
#     station.GetInputPort("mobile_iiwa.desired_state"),
#     "mobile_iiwa.desired_state"
# )

q_traj_system = builder.AddSystem(TrajectorySource(q_traj_book_grasp))
g_traj_system = builder.AddSystem(TrajectorySource(traj_wsg_book_grasp))

state_interpolator = builder.AddSystem(StateInterpolatorWithDiscreteDerivative(num_positions=10, time_step=0.1))

# builder.Connect(
#     q_traj_system.get_output_port(), station.GetInputPort("mobile_iiwa.desired_state")
# )

builder.Connect(
    q_traj_system.get_output_port(), state_interpolator.GetInputPort("position")
)

builder.Connect(
    state_interpolator.GetOutputPort("state"), station.GetInputPort("mobile_iiwa.desired_state")
)

builder.Connect(
    g_traj_system.get_output_port(), station.GetInputPort("wsg.position")
)


visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)
diagram = builder.Build()
#diagram.set_name("plant and scene_graph")
diagram_context = diagram.CreateDefaultContext()
scene_graph = diagram.GetSubsystemByName("station").GetSubsystemByName("scene_graph")
sg_context = scene_graph.GetMyContextFromRoot(diagram_context)
filterCollsionGeometry(scene_graph, sg_context)

simulator = Simulator(diagram)
#fix_input_port(diagram, simulator)
visualizer.StartRecording(False)
simulator.AdvanceTo(1.0)
#Run this to see what happened
visualizer.PublishRecording()

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



In [34]:
from grasp.grasping import *
from hwstation.utils import visualize_frame

In [77]:
def get_shelf_gripper_traj(diagram: Diagram, diagram_context: Context, shelf_label: str, row_idx: int, col_idx: int):
    placement_frame = get_shelf_placement_frame(diagram, diagram_context, shelf_label, row=row_idx, column=col_idx)
    world_to_gripper = RigidTransform(RotationMatrix(np.array([[0,1,0],[1,0,0],[0,0,-1]])))
    factor = 5
    if shelf_label in ['E', 'F']:
        factor = 4
    placement_frame = RigidTransform(placement_frame.rotation(),
    p=placement_frame.translation() + np.array([-SHELF_WIDTH/factor,0,0.05])) #np.array([-SHELF_WIDTH/5,0,-0.05])) 
    placement_frame = placement_frame.multiply(world_to_gripper)
    visualize_frame("placement_frame", meshcat, placement_frame)
    scale_factor_away = 1.25
    if shelf_label in ["E", "F", "B"]:
        scale_factor_away = 1.5
    preplacement_frame = compute_pre_placement_frame(placement_frame, scale_factor_away) 
    visualize_frame("preplacement_frame", meshcat, preplacement_frame)
    return preplacement_frame, placement_frame

def clip_to_joint_limits(q_config, joint_limits, tol=0.01):
    q_config = q_config.copy()
    upper_mask = (q_config>joint_limits[:,1])
    lower_mask = (q_config<joint_limits[:,0])
    upper_distance_mask = np.abs(q_config-joint_limits[:,1])
    lower_distance_mask = np.abs(q_config-joint_limits[:,0])
    upper_mask = (upper_mask & (upper_distance_mask<tol))
    lower_mask = (lower_mask & (lower_distance_mask<tol))
    q_config[upper_mask] = joint_limits[upper_mask,1]
    q_config[lower_mask] = joint_limits[lower_mask,0]
    return q_config

# what's different about this function is that now we try to line up with the book but be away from the table
def find_shelf_drop_grasp(manipSim: ManipulationStationSim, diagram: Diagram, diagram_context: Context, q0_init: np.ndarray, gripper_setpoint: float ,placement_frame: RigidTransform, preplacement_frame: RigidTransform):
    joint_limits = get_joint_limits(diagram.GetSubsystemByName("station").GetSubsystemByName("plant"))
    while True:
        manipSim.SetStationConfiguration(q0_home.copy(), gripper_setpoint)
        #q0 = get_iiwa_joint_state(diagram, diagram_context)
        q0 = q0_home.copy()
        ik_solver = IKSolver()
        q_middle_goal, optimal = ik_solver.solve(preplacement_frame, q_guess=q0)
        #Clip to join limits (may be cause by numerical errors)
        # q0 = clip_to_joint_limits(q0, joint_limits)
        q0 = q0_home.copy()
        q_middle_goal = clip_to_joint_limits(q_middle_goal, joint_limits)

        start_collision = manipSim.ExistsCollision(q0,gripper_setpoint)
        middle_collision = manipSim.ExistsCollision(q_middle_goal,gripper_setpoint)
        
        if (not start_collision) and (not middle_collision):
            manipSim.SetStationConfiguration(q_middle_goal, gripper_setpoint)
            q_middle = get_iiwa_joint_state(diagram, diagram_context)
            ik_solver = IKSolver()
            q_guess = q_middle.copy()
            #q_guess[0] = q_guess[0] - SHELF_WIDTH/2
            q_goal, optimal = ik_solver.solve(placement_frame, q_guess=q_guess)
            #Clip to join limits (may be cause by numerical errors)
            q_goal = clip_to_joint_limits(q_goal, joint_limits)
            q_middle = clip_to_joint_limits(q_middle, joint_limits)

            middle_collision = manipSim.ExistsCollision(q_middle,gripper_setpoint)
            final_collision = manipSim.ExistsCollision(q_goal,gripper_setpoint)
            if (not middle_collision) and (not final_collision):
                break
    return q0, q_middle_goal, q_goal

In [78]:
def find_path_to_shelf(q0_way_point: np.ndarray, q_middle_goal: np.ndarray, q_goal: np.ndarray, manipSim: ManipulationStationSim, diagram: Diagram, diagram_context: Context, max_rrt_iter: int = 500, joint_limits_dict: dict = {}, gripper_setpoint: float = 0.1):
    path_found = False 
    while not path_found:
        iiwa_problem = IiwaProblem(
            q_start=q0_way_point,
            q_goal=q_middle_goal,
            gripper_setpoint=gripper_setpoint,
            collision_checker=manipSim,
            debug=True,
            joint_limits_dict=joint_limits_dict,
            x_limits=(-1.0,9.0),
            y_limits=(-1.0,9.0),
        )

        path = rrt_planning(iiwa_problem, max_rrt_iter, 0.20, jump_to_goal=True)
        if path is None: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
            print("RESTART RRT")
            continue
        print("Finished first path")
        further_iiwa_problem = IiwaProblem(
            q_start=q_middle_goal,
            q_goal=q_goal,
            gripper_setpoint=gripper_setpoint,
            collision_checker=manipSim,
            debug=True,   
            x_limits=(-1.0,9.0),
            y_limits=(-1.0,9.0),
        )
        further_path = rrt_planning(further_iiwa_problem, max_rrt_iter, 0.20, jump_to_goal=True)
        if further_path is None: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
            print("RESTART RRT due to further path failing")
            continue

        path.extend(further_path)
        
        path_found = True
    return path

# Configure this path and obtain joint and wsg_trajectories
def configure_grasping_path(path_arr, start_time, book_placement_time):
    path_arr = np.array(path_arr).T
    wait_at_grasp = len(path_arr[0])
    grasp_pose = path_arr[:, -1]
    grasp_pose = np.tile(grasp_pose, (wait_at_grasp,1)).T
    reversed_path = path_arr[:,::-1]

    # arrive at grasp pose and wait
    complete_path = np.concatenate([path_arr,grasp_pose],axis=1)

    # return from grasp pose
    complete_path = np.concatenate([complete_path,reversed_path],axis=1)

    t_lst = np.linspace(start_time, book_placement_time, complete_path.shape[1])
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, complete_path)

    pre_pick_time_idx = len(path_arr[0])
    post_pick_time_idx = (int)(pre_pick_time_idx + wait_at_grasp)

    opened = np.array([0.1])
    closed = np.array([0.0])

    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
        [t_lst[0], t_lst[pre_pick_time_idx]],
        np.hstack([[closed], [closed]]),
    )
    traj_wsg_command.AppendFirstOrderSegment(t_lst[post_pick_time_idx], opened)
    traj_wsg_command.AppendFirstOrderSegment(t_lst[-1], opened)

    return q_traj, traj_wsg_command

In [79]:
q_traj_book_place_list = []
traj_wsg_book_place_list = []

In [80]:
q_traj_book_place_list = []
traj_wsg_book_place_list = []
for idx in range(N_BOOKS_TO_GRASP):
    print("planning going to shelf: ", shelf_labels[idx])
    shelf_label = shelf_labels[idx]
    row_idx = shelf_pos_list[idx][0]
    col_idx = shelf_pos_list[idx][1]

    print("get_shelf_gripper_traj")
    preplacement_frame, placement_frame = get_shelf_gripper_traj(diagram, diagram_context,shelf_label,row_idx,col_idx)
    
    print("get_iiwa_joint_state")
    q0_post_grasping = get_iiwa_joint_state(diagram, diagram_context)
    q0_post_grasping = q0_home.copy()
    manipSim = ManipulationStationSim(diagram, diagram_context)

    print("find_shelf_drop_grasp")
    q0_post_grasping, q_middle_goal, q_goal = find_shelf_drop_grasp(manipSim, diagram, diagram_context,q0_post_grasping,0.1,placement_frame, preplacement_frame)
    q0_post_grasping = q0_home.copy()
    q_middle_goal[4:] = q0_post_grasping[4:].copy()
    joint_limits_dict = dict(zip([f"iiwa_joint_{idx}" for idx in range(2,8)],[[q0_index-0.01,q0_index+0.01] for q0_index in q0_post_grasping[4:]]))
    joint_limits_test = get_joint_limits(diagram_plan.GetSubsystemByName("station").GetSubsystemByName("plant"),joint_limits_dict=joint_limits_dict)

    # add waypoints
    waypoint_path = []
    q0_way_point = q0_post_grasping.copy()
    if shelf_label == "D":
        q0_way_point[0] += ROW_DIST_SHELF
        waypoint_path = np.linspace(q0_post_grasping,q0_way_point,70).tolist()


    if shelf_label == "F":
        q0_way_point[0] += 2*ROW_DIST_SHELF
        waypoint_path = np.linspace(q0_post_grasping,q0_way_point,140).tolist()

    print("find_path_to_shelf")
    max_rrt_iter = 1000
    if shelf_label == ["A"]:
        max_rrt_iter = 200
    path = find_path_to_shelf(q0_way_point, q_middle_goal, q_goal, manipSim, diagram_plan, diagram_plan_context, max_rrt_iter=max_rrt_iter, joint_limits_dict=joint_limits_dict, gripper_setpoint=0.1)
    waypoint_path.extend(path)
    path = waypoint_path

    print("configure_grasping_path")
    q_traj_book_place, traj_wsg_book_place = configure_grasping_path(path,start_time=time_point_list[idx+1][0], book_placement_time=time_point_list[idx+1][1])
    
    #Append trajectories:
    q_traj_book_place_list.append(q_traj_book_place)
    traj_wsg_book_place_list.append(traj_wsg_book_place)

planning going to shelf:  A
get_shelf_gripper_traj
get_iiwa_joint_state
find_shelf_drop_grasp
find_path_to_shelf
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
Checking whether are close enough to goal to connect. Almost there.
Finished first path
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal to connect. Almost there.
Checking whether are close enough to goal t

KeyboardInterrupt: 

In [ ]:
q_traj_final_list = []
traj_wsg_final_list = []
for idx in range(N_BOOKS_TO_GRASP):
    q_traj_final_list.append(q_traj_grasp_list[idx])
    q_traj_final_list.append(q_traj_book_place_list[idx])

    traj_wsg_final_list.append(traj_wsg_grasp_list[idx])
    traj_wsg_final_list.append(traj_wsg_book_place_list[idx])

In [73]:
from pydrake.all import StackedTrajectory, CompositeTrajectory
q_traj_final = CompositeTrajectory(q_traj_final_list)
traj_wsg_final = CompositeTrajectory(traj_wsg_final_list)

In [74]:
save_traj(q_traj_final,Q_TRAJECTORY_FPATH,grid_points=10000)
q_traj_reloaded = load_traj(Q_TRAJECTORY_FPATH)
for t in np.random.uniform(size=10000,low=q_traj_final.start_time(), high=q_traj_final.end_time()):
    np.allclose(q_traj_final.value(t),q_traj_reloaded.value(t))


(10000, 11)


In [75]:
save_traj(traj_wsg_final,WSG_TRAJECTORY_FPATH,grid_points=10000)
traj_wsg_reloaded = load_traj(WSG_TRAJECTORY_FPATH)
for t in np.random.uniform(size=10000,low=traj_wsg_final.start_time(), high=traj_wsg_final.end_time()):
    np.allclose(traj_wsg_reloaded.value(t),traj_wsg_final.value(t))

(10000, 2)


In [ ]:
Q_TRAJECTORY_FPATH = "trajectories/6_q_book_traj"
WSG_TRAJECTORY_FPATH = "trajectories/6_wsg_book_traj"

## Saving trajectories individually

In [83]:
for idx, q_traj in enumerate(q_traj_final_list):
    Q_TRAJECTORY_FPATH = "trajectories/6_q_book_traj_" + str(idx)
    save_traj(q_traj,Q_TRAJECTORY_FPATH,grid_points=10000)
    q_traj_reloaded = load_traj(Q_TRAJECTORY_FPATH)
    for t in np.random.uniform(size=10000,low=q_traj.start_time(), high=q_traj.end_time()):
        np.allclose(q_traj.value(t),q_traj.value(t))

(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)
(10000, 11)


In [84]:
for idx, wsg_traj in enumerate(traj_wsg_final_list):
    WSG_TRAJECTORY_FPATH = "trajectories/6_wsg_book_traj_" + str(idx)
    save_traj(wsg_traj,WSG_TRAJECTORY_FPATH,grid_points=10000)
    wsg_traj_reloaded = load_traj(WSG_TRAJECTORY_FPATH)
    for t in np.random.uniform(size=10000,low=wsg_traj.start_time(), high=wsg_traj.end_time()):
        np.allclose(wsg_traj.value(t),wsg_traj.value(t))

(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
(10000, 2)
